In [65]:
import numpy as np
import os
import re
import cv2

from collections import Counter

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns


In [66]:
def detect_belt(bi_images):
    try:
        # Convert to HSV color space for better color edge detection
        hsv = cv2.cvtColor(bi_images, cv2.COLOR_BGR2HSV)

        # Calculate gradients (Sobel operator) for each channel
        sobelx = cv2.Sobel(hsv[:, :, 0], cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(hsv[:, :, 1], cv2.CV_64F, 0, 1, ksize=3)

        # Combine gradients (magnitude)
        magnitude, angle = cv2.cartToPolar(sobelx, sobely, angleInDegrees=True)

        # Apply threshold for edge detection
        threshold = 0.1 * cv2.norm(magnitude, cv2.NORM_INF)
        edges = magnitude > threshold

        # Apply morphological closing to enhance edges (optional)
        edges = cv2.morphologyEx(np.uint8(edges), cv2.MORPH_CLOSE, kernel=np.ones((3, 3), np.uint8))

        # Find contours (connected components)
        contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter edges by height
        filtered_edges = bi_images.copy()
        image_height = bi_images.shape[0]
        xcrop_end = bi_images.shape[1]  # Default to image width

        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            if h > (0.7 * image_height):
                cv2.rectangle(filtered_edges, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw green rectangle
                xcrop_end = w

    except Exception as e:
        print(f"An error occurred: {e}")
        xcrop_end = bi_images.shape[1]

    return xcrop_end

def subtract_images(background_path, foreground_path):
    # Load the images
    background = background_path #cv2.imread(background_path)
    foreground = foreground_path #cv2.imread(foreground_path)

    # Check if images are loaded successfully
    if background is None:
        print(f"Error: Unable to load image at {background_path}")
        return None
    if foreground is None:
        print(f"Error: Unable to load image at {foreground_path}")
        return None

    # Ensure both images have the same size (resize if necessary)
    if background.shape != foreground.shape:
        foreground = cv2.resize(foreground, (background.shape[1], background.shape[0]))

    # Perform image subtraction
    subtracted_image = cv2.absdiff(foreground, background)

    # Convert to grayscale
    subtracted_gray = cv2.cvtColor(subtracted_image, cv2.COLOR_BGR2GRAY)

    # Thresholding to create a mask
    _, mask = cv2.threshold(subtracted_gray, 10, 255, cv2.THRESH_BINARY)

    # Apply morphological opening to remove noise (small dots)
    kernel = np.ones((3, 3), np.uint8)
    clean_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Invert the mask
    clean_mask = 255 - clean_mask

    # Apply the mask to make the background white
    subtracted_image[mask == 0] = [0, 0, 0]

    return subtracted_image

def extract_gcode_text(gi_image):
    # Convert image to grayscale
    gray = cv2.cvtColor(gi_image, cv2.COLOR_BGR2GRAY)

    # Apply threshold to extract red text (assuming red channel is dominant)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

    # Invert the image
    inverted = cv2.bitwise_not(thresh)

    # Convert inverted image to BGR format for visualization
    inverted_bgr = cv2.cvtColor(inverted, cv2.COLOR_GRAY2BGR)

    return inverted_bgr

def resize_and_pad(image, target_size):
    h, w = image.shape[:2]
    sh, sw = target_size

    # Scale the image
    aspect = w / h
    if aspect > 1:
        new_w = sw
        new_h = int(new_w / aspect)
    else:
        new_h = sh
        new_w = int(new_h * aspect)

    resized = cv2.resize(image, (new_w, new_h))

    # Pad the image
    delta_w = sw - new_w
    delta_h = sh - new_h
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    color = [0, 0, 0]  # Color for padding (black)
    padded = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    return padded

def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Use cv2.IMREAD_GRAYSCALE for grayscale
    image = resize_and_pad(image, target_size)
    image = image.astype('float32') / 255.0  # Normalize to [0, 1]
    image = np.expand_dims(image, axis=-1)   # Add channel dimension
    return image



def build_cnn(input_shape):
    model = tf.keras.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu')
    ])
    return model

def build_siamese_network():
    input_a = Input(shape=(224, 224, 1))
    input_b = Input(shape=(224, 224, 1))
    
    cnn = build_cnn((224, 224, 1))
    
    processed_a = cnn(input_a)
    processed_b = cnn(input_b)
    
    # Calculate the L1 distance between the feature vectors
    distance = Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([processed_a, processed_b])
    
    # Output layer: binary classification (correct or faulty)
    output = Dense(1, activation='sigmoid')(distance)
    
    model = Model([input_a, input_b], output)
    return model

In [67]:
blanc_path = '/Users/soukh2/Desktop/Wunderpen_Shubham/test_images/Blanc_25.png' ## path for blanc image
written_path = '/Users/soukh2/Desktop/Wunderpen_Shubham/test_images/test2.png' ## path for written image
gcode_path = '/Users/soukh2/Desktop/Wunderpen_Shubham/test_images/25.png' ## path for gcode image


In [68]:
## location to store the images
output_dir_correct = './test_content'
if not os.path.exists(output_dir_correct):
    os.makedirs(output_dir_correct)    

output_dir_correct_gcode = './test_content_gcode'
if not os.path.exists(output_dir_correct_gcode):
    os.makedirs(output_dir_correct_gcode) 

In [69]:
bi_image = cv2.imread(blanc_path)
wi_image = cv2.imread(written_path)
gi_image = cv2.imread(gcode_path)

### remmoving the rollers from the frame
bi_images = bi_image[:,150:,:]
wi_images = wi_image[:,150:,:]

xcrop_end = detect_belt(bi_images)

bi_images = bi_images[:, :xcrop_end, :]
wi_images = wi_images[:, :xcrop_end, :]
#print(bi_images.shape, wi_images.shape)

result = subtract_images(bi_images, wi_images)
result = cv2.rotate(result, cv2.ROTATE_90_CLOCKWISE)
# Convert inverted image to BGR format for visualization
inverted_bgr = extract_gcode_text(gi_image)

### saving images

mpimg.imsave(f'./test_content/written.png', result)
mpimg.imsave(f'./test_content_gcode/gcode.png', inverted_bgr)



In [70]:
input_shape = (224, 224, 1)
siamese_network = build_siamese_network()
siamese_network.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [71]:
checkpoint_dir = './checkpoints'
weights_path = os.path.join(checkpoint_dir, f'best_model.weights.h5')
siamese_network.load_weights(weights_path)

In [72]:
image_path_written = './test_content/written.png'
image_path_gcode =  './test_content_gcode/gcode.png'

In [73]:
# Preprocess the new images
test_image_written = preprocess_image(image_path_written)
test_image_gcode = preprocess_image(image_path_gcode)

# Add batch dimension
test_image_written = np.expand_dims(test_image_written, axis=0)
test_image_gcode = np.expand_dims(test_image_gcode, axis=0)


In [74]:
# Get the prediction
prediction = siamese_network.predict([test_image_written, test_image_gcode])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


In [75]:
if prediction[0][0] >= 0.5:
    print("The test image is predicted to be correct.")
else:
    print("The test image is predicted to be faulty.")

The test image is predicted to be faulty.
